## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-20-03-54-26 +0000,nyt,"White House Presses Meeting, Security Guarante...",https://www.nytimes.com/live/2025/08/19/us/tru...
1,2025-08-20-03-54-25 +0000,nyt,"Securing Postwar Ukraine, Even With Trump’s Pl...",https://www.nytimes.com/2025/08/19/world/europ...
2,2025-08-20-03-51-54 +0000,startribune,"Joe Ryan suffers shortest start of season, Twi...",https://www.startribune.com/minnesota-twins-lo...
3,2025-08-20-03-36-51 +0000,bbc,"US in talks over 10% Intel stake, White House ...",https://www.bbc.com/news/articles/c620gg77epxo...
4,2025-08-20-03-26-06 +0000,nypost,Investigators seize car belonging to father of...,https://nypost.com/2025/08/19/us-news/investig...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2303/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,51
11,ukraine,26
216,deal,16
6,russia,14
298,zelensky,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
73,2025-08-19-22-54-43 +0000,nypost,Russia launches largest strike on Ukraine in w...,https://nypost.com/2025/08/19/world-news/russi...,132
242,2025-08-19-14-07-00 +0000,wsj,"After months of effort, President Trump says a...",https://www.wsj.com/world/trump-plans-to-get-p...,129
126,2025-08-19-20-30-23 +0000,latimes,Contributor: Trump's Russia and Ukraine summit...,https://www.latimes.com/opinion/story/2025-08-...,122
133,2025-08-19-20-23-41 +0000,bbc,Kremlin plays down Zelensky talks as Trump war...,https://www.bbc.com/news/articles/cn92e52rpjxo...,114
268,2025-08-19-10-25-29 +0000,nypost,Putin and Zelensky could meet within next two ...,https://nypost.com/2025/08/19/world-news/putin...,111


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
73,132,2025-08-19-22-54-43 +0000,nypost,Russia launches largest strike on Ukraine in w...,https://nypost.com/2025/08/19/world-news/russi...
89,67,2025-08-19-21-56-00 +0000,wsj,"Flight Attendants Win Boarding Pay, Four-Year ...",https://www.wsj.com/business/airlines/air-cana...
68,55,2025-08-19-23-01-18 +0000,nypost,Curtis Sliwa’s quality-of-life crackdown makes...,https://nypost.com/2025/08/19/opinion/i-agree-...
140,48,2025-08-19-19-55-47 +0000,nypost,Texas Dem Rep. Nicole Collier slept in state H...,https://nypost.com/2025/08/19/us-news/texas-de...
228,38,2025-08-19-14-54-58 +0000,nyt,Netanyahu Faces Pressure From Far Right Over N...,https://www.nytimes.com/2025/08/19/world/middl...
81,38,2025-08-19-22-29-36 +0000,nypost,Socialist Zohran Mamdani holds private meeting...,https://nypost.com/2025/08/19/us-news/zohran-m...
206,37,2025-08-19-16-34-37 +0000,nypost,Memoir from Oct. 7 hostage Eli Sharabi will re...,https://nypost.com/2025/08/19/world-news/memoi...
133,36,2025-08-19-20-23-41 +0000,bbc,Kremlin plays down Zelensky talks as Trump war...,https://www.bbc.com/news/articles/cn92e52rpjxo...
201,33,2025-08-19-17-09-45 +0000,nypost,China’s Pop Mart sees profit soar 400% as Labu...,https://nypost.com/2025/08/19/business/chinas-...
45,32,2025-08-20-00-40-22 +0000,nypost,Husband of woman found dead in national forest...,https://nypost.com/2025/08/19/us-news/husband-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
